In [156]:
import numpy as np
import pandas as pd
from scipy import stats

#Calcula el % de valores por columna.
def valores_nulos_col(df):
    n_records=len(df)
    for columna in df:
        print ("{}|{}|{}".format(
            df[columna].name,len(df[df[columna].isnull()])/(1*n_records),df[columna].dtype
               ))
        
#Saco la lista de los valores fuera distribucion(outlier).
def detect_outlier(data_1):
    outliers=[]
    threshold=3
    mean_1 = np.mean(data_1)
    std_1 =np.std(data_1)
    for y in data_1:
        z_score= (y - mean_1)/std_1 
        if np.abs(z_score) > threshold:
            outliers.append(y)
    return outliers

#Saca el numero de valores que encuentran fuera de distribucion 
def outliers_col(df):
    for columna in df:
        if df[columna].dtype != np.object:
            n_outliers =len(df[np.abs(stats.zscore(df[columna]))>3])
            print ("{}|{}|{}".format(
                df[columna].name,
                n_outliers,
                df[columna].dtype
            ))
#Busco los valores fuera de la distrucion y los cambio por media corta a 30%.
def cambiar_outlier(data_1):
    threshold=3
    mean_1 = np.mean(data_1)
    mean_30 = stats.trim_mean(data_1, 0.300);
    std_1 =np.std(data_1)
    for y in data_1:
        z_score= (y - mean_1)/std_1 
        if np.abs(z_score) > threshold:
            data_1=data_1.replace(y,mean_30)
    return data_1            
            
#Funcion que valida si dato es un float, 
#Si no es float lo vuelve 0 para que genere problemas con convercion de formato
def is_float(value):
  try:
    value= float("{0:.4f}".format(value))
    return value
  except:
    return 0.0



In [157]:
import pandas as pd

Ruta="../"

#Datos La información sobre motos y vehículos particulares de Diagnóstico Automotor del Valle

Tasa_de_motorizacion_general = pd.read_excel(Ruta+'Originales/Tasa_de_motorizacion_general.xlsx',sheet_name='Tasa_de_motorizacion_general', header=0)
Tasa_de_motorizacion_general=Tasa_de_motorizacion_general.rename(columns={
  "Año":"Año",
    "Motos particulares":"Motos_particulares",
    "Vehículos particulares":"Vehiculos_particulares",
    "Motos + Vehículos particulares":"Motos_vehiculos_particulares",
    "Tasa de Crecimiento General":"Tasa_crecimiento_general"
})

Tasa_de_motorizacion_general.head(19)

,Año,Motos_particulares,Vehiculos_particulares,Motos_vehiculos_particulares,Tasa_crecimiento_general
0,2000,55462.0,139952.0,195414.0,NaN
1,2001,56045.0,145394.0,201439.0,0.030832
2,2002,56498.0,152672.0,209170.0,0.038379
3,2003,57038.0,160323.0,217361.0,0.039160
4,2004,58267.0,168837.0,227104.0,0.044824
5,2005,60679.0,179831.0,240510.0,0.059030
6,2006,64309.0,194554.0,258863.0,0.076309
7,2007,66650.0,212359.0,279009.0,0.077825
8,2008,69871.0,227705.0,297576.0,0.066546
9,2009,77256.0,239913.0,317169.0,0.065842


In [158]:
import pandas as pd
from scipy import stats # importando scipy.stats
from pandas import ExcelWriter
from pandas import ExcelFile

Ruta="../"


#Revision de los datos de la estacion la ERA 2017.


eraDatos = pd.read_excel(Ruta+'Originales/2.ERA.xls',sheet_name='2010 (ug-m3)', header=0)

#Agrupo los datos de contaminacion y Tasa_de_motorizacion_general.

eraDatos['Tasa_crecimiento_general']=Tasa_de_motorizacion_general.iloc[10,4]
#Esta año no tenia dos sensores se remplaza por media de la flora
eraDatos['H2S (ug/m3)']=0.0
eraDatos['O3  (ug/m3)']=17.64

eraDatos=eraDatos.rename(columns={
  "Fecha & Hora":"Fecha",
    "H2S (ug/m3)":"H2S",
  "PM10 (ug/m3)":"PM10",
    "SO2  (ug/m3)":"SO2",
    "NO2  (ug/m3)":"NO2",
    "CO  (ug/m3)":"CO",
    "O3  (ug/m3)":"O3",
    "Vel Viento  (m/s)":"VelVien",
    "Dir Viento (Grados)":"DirVien",
    "Temperatura (C°)":"Temp", 
    "Humedad (%)":"Humedad",
    "Radiacion Solar (Watt/M2)":"RSolar", 
    "Lluvia (mm)":"Lluvia" 
})

In [159]:
#Crea una columna con los numero de nulos.
eraDatos['N_NA']=eraDatos.isnull().sum(axis=1)

#Solo dejo los registros con menos de 3 nulos.
eraDatos=eraDatos[eraDatos.N_NA<3]

#Borro la columna temporal.
eraDatos=eraDatos.drop(['N_NA'], axis=1)
eraDatos=eraDatos.drop(['Fecha'], axis=1)

In [160]:
eraDatos.shape

(7616, 13)

In [161]:
eraDatos.head(4)

,PM10,SO2,NO2,CO,VelVien,DirVien,Temp,Humedad,RSolar,Lluvia,Tasa_crecimiento_general,H2S,O3
812,NaN,6.021808,NaN,1101.839779,3.4,267.8,28.1,59.8,0.0,0.0,0.097415,0.0,17.64
813,NaN,6.089257,NaN,1018.621784,3.0,277.3,27.9,60.1,0.0,0.0,0.097415,0.0,17.64
814,NaN,4.550680,NaN,763.458887,2.2,271.8,27.7,61.7,0.0,0.0,0.097415,0.0,17.64
815,NaN,3.992419,NaN,635.792864,1.6,269.3,27.2,64.3,0.0,0.0,0.097415,0.0,17.64


In [162]:
# Cambio los N/A por cero

eraDatos['PM10'].fillna(0, inplace=True)
eraDatos['SO2'].fillna(0, inplace=True)
eraDatos['NO2'].fillna(0, inplace=True)
eraDatos['CO'].fillna(0, inplace=True)
eraDatos['H2S'].fillna(0, inplace=True)
eraDatos['VelVien'].fillna(0, inplace=True)
eraDatos['DirVien'].fillna(0, inplace=True)
eraDatos['Temp'].fillna(0, inplace=True)
eraDatos['Humedad'].fillna(0, inplace=True)
eraDatos['RSolar'].fillna(0, inplace=True)
eraDatos['Lluvia'].fillna(0, inplace=True)

In [163]:

#Aplico la funcion is_float en cada unos de la columnas.

eraDatos['NO2'] = eraDatos.apply(lambda x:is_float(x['NO2']), axis=1)
eraDatos['PM10'] = eraDatos.apply(lambda x:is_float(x['PM10']), axis=1)
eraDatos['SO2'] = eraDatos.apply(lambda x:is_float(x['SO2']), axis=1)
eraDatos['CO'] = eraDatos.apply(lambda x:is_float(x['CO']), axis=1)
eraDatos['H2S'].fillna(0, inplace=True)
eraDatos['DirVien'] = eraDatos.apply(lambda x:is_float(x['DirVien']), axis=1)
eraDatos['Humedad'] = eraDatos.apply(lambda x:is_float(x['Humedad']), axis=1)
eraDatos['Lluvia'] = eraDatos.apply(lambda x:is_float(x['Lluvia']), axis=1)
eraDatos['RSolar'] = eraDatos.apply(lambda x:is_float(x['RSolar']), axis=1)
eraDatos['Temp'] = eraDatos.apply(lambda x:is_float(x['Temp']), axis=1)
eraDatos['VelVien'] = eraDatos.apply(lambda x:is_float(x['VelVien']), axis=1)
eraDatos['Tasa_crecimiento_general'] = eraDatos.apply(lambda x:is_float(x['Tasa_crecimiento_general']), axis=1)

In [164]:
#Vuelvo float32 todas las columnas.

eraDatos['PM10'] = eraDatos['PM10'].astype('float32')
eraDatos['SO2'] = eraDatos['SO2'].astype('float32')
eraDatos['DirVien'] = eraDatos['DirVien'].astype('float32')
eraDatos['Humedad'] = eraDatos['Humedad'].astype('float32')
eraDatos['Lluvia'] = eraDatos['Lluvia'].astype('float32')
eraDatos['RSolar'] = eraDatos['RSolar'].astype('float32')
eraDatos['Temp'] = eraDatos['Temp'].astype('float32')
eraDatos['VelVien'] = eraDatos['VelVien'].astype('float32')
eraDatos['CO'] =  eraDatos['CO'].astype('float32')
eraDatos['Tasa_crecimiento_general'] =  eraDatos['Tasa_crecimiento_general'].astype('float32')

In [165]:

eraDatos['Temp']=cambiar_outlier(eraDatos['Temp'])
eraDatos['PM10']=cambiar_outlier(eraDatos['PM10'])
eraDatos['SO2']=cambiar_outlier(eraDatos['SO2'])
eraDatos['DirVien']=cambiar_outlier(eraDatos['DirVien'])
eraDatos['Humedad']=cambiar_outlier(eraDatos['Humedad'])
eraDatos['Lluvia']=cambiar_outlier(eraDatos['Lluvia'])
eraDatos['RSolar']=cambiar_outlier(eraDatos['RSolar'])
eraDatos['VelVien']=cambiar_outlier(eraDatos['VelVien'])
eraDatos['CO']=cambiar_outlier(eraDatos['CO'])

In [167]:
#CO	DirVien	H2S	Humedad	Lluvia	NO2	O3	PM10	RSolar	SO2	Tasa_crecimiento_general	Temp	VelVien

eraDatos.columns

Index(['PM10', 'SO2', 'NO2', 'CO', 'VelVien', 'DirVien', 'Temp', 'Humedad',
       'RSolar', 'Lluvia', 'Tasa_crecimiento_general', 'H2S', 'O3'],
      dtype='object')

In [170]:
#Ordeno la columnas para que tenga la misma forma del modelo.
#PM10 0
#DirVien 5
#H2S  12
#Humedad 7
#Lluvia 9
#NO2 2
#O3 13
#CO 3
#RSolar 8
#SO2 1
#Tasa_crecimiento_general 11
#Temp 6
#VelVien

new_order = [0,5,11,7,9,2,12,3,8,1,10,6,4]
eraDatos = eraDatos[eraDatos.columns[new_order]]

eraDatos.describe()

,PM10,DirVien,H2S,Humedad,Lluvia,NO2,O3,CO,RSolar,SO2,Tasa_crecimiento_general,Temp,VelVien
count,7616.000000,7616.000000,7616.0,7616.000000,7616.000000,7616.000000,7.616000e+03,7616.000000,7616.000000,7616.000000,7616.000000,7616.000000,7616.000000
mean,26.908112,170.555344,0.0,71.675407,0.507670,29.193293,1.764000e+01,1362.113037,161.597275,11.976591,0.097407,24.495308,1.282723
std,20.520992,89.296349,0.0,13.993754,3.525591,69.889368,1.812003e-13,695.949219,237.679031,12.255437,0.000007,3.100022,1.005162
min,0.000000,0.200000,0.0,0.917100,0.000000,0.000000,1.764000e+01,0.000000,0.000000,0.000000,0.097400,0.917400,0.000000
25%,11.000000,97.099998,0.0,59.799999,0.000000,0.000000,1.764000e+01,871.143112,0.000000,4.759575,0.097400,22.000000,0.600000
50%,23.000000,166.699997,0.0,72.800003,0.000000,26.027600,1.764000e+01,1280.327759,3.350000,9.114000,0.097400,24.100000,1.000000
75%,39.000000,257.025002,0.0,82.900002,0.000000,44.735775,1.764000e+01,1769.243530,284.275009,15.793900,0.097400,26.700001,1.500000
max,169.000000,359.899994,0.0,100.000000,110.489998,5705.000000,1.764000e+01,3613.295654,929.400024,159.140594,0.097400,33.900002,6.300000


In [171]:
eraDatos.to_excel("eraDatos.xlsx", sheet_name='eraDatos',encoding='UTF-8') #Se crea el archivo definitivo.

In [172]:
eraDatos.head(24)

,PM10,DirVien,H2S,Humedad,Lluvia,NO2,O3,CO,RSolar,SO2,Tasa_crecimiento_general,Temp,VelVien
812,0.0,267.799988,0.0,59.799999,0.0,0.0,17.64,1101.839844,0.000000,6.021800,0.0974,28.100000,3.4
813,0.0,277.299988,0.0,60.099998,0.0,0.0,17.64,1018.621826,0.000000,6.089300,0.0974,27.900000,3.0
814,0.0,271.799988,0.0,61.700001,0.0,0.0,17.64,763.458923,0.000000,4.550700,0.0974,27.700001,2.2
815,0.0,269.299988,0.0,64.300003,0.0,0.0,17.64,635.792908,0.000000,3.992400,0.0974,27.200001,1.6
816,0.0,39.700001,0.0,71.000000,0.0,0.0,17.64,588.573120,0.000000,3.325900,0.0974,25.700001,0.5
817,0.0,138.800003,0.0,72.400002,0.0,0.0,17.64,631.564087,0.000000,2.757700,0.0974,25.299999,0.4
818,0.0,161.800003,0.0,74.199997,0.0,0.0,17.64,871.538086,0.000000,3.986800,0.0974,24.900000,0.5
819,0.0,159.500000,0.0,75.599998,0.0,0.0,17.64,778.751587,0.000000,3.510000,0.0974,24.400000,0.7
820,0.0,191.100006,0.0,73.300003,0.0,0.0,17.64,720.279480,0.000000,4.157800,0.0974,24.299999,0.3
821,0.0,191.800003,0.0,74.699997,0.0,0.0,17.64,720.037292,0.000000,4.626900,0.0974,24.000000,0.5


In [173]:
eraDatos.shape

(7616, 13)